## Beta - Bank ##

`Beta Bank está enfrentando un problema creciente de pérdida de clientes. Cada mes, más usuarios abandonan sus servicios, lo cual representa un costo importante para la institución. En respuesta, este proyecto tiene como objetivo desarrollar un modelo de machine learning capaz de predecir si un cliente abandonará el banco próximamente. Para ello, se utilizará un conjunto de datos con información demográfica y financiera de 10,000 clientes, incluyendo su historial de actividad. El modelo será evaluado principalmente con la métrica F1-score, buscando alcanzar un valor mínimo de 0.59 para su aprobación. Además, se analizará la métrica AUC-ROC para comparar su desempeño en términos de clasificación. A lo largo del proceso se abordará el problema del desbalance de clases y se implementarán distintas técnicas para mejorarlo.`

Descripción de los datos
Puedes encontrar los datos en el archivo  /datasets/Churn.csv file. Descarga el conjunto de datos.

Características: 

- RowNumber: índice de cadena de datos
- CustomerId: identificador de cliente único
- Surname: apellido
- CreditScore: valor de crédito
- Geography: país de residencia
- Gender: sexo
- Age: edad
- Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
- Balance: saldo de la cuenta
- NumOfProducts: número de productos bancarios utilizados por el cliente
- HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
- IsActiveMember: actividad del cliente (1 - sí; 0 - no)
- EstimatedSalary: salario estimado

Objetivo:

- Exited: El cliente se ha ido (1 - sí; 0 - no)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

In [2]:
# Cargar el archivo
df = pd.read_csv('/datasets/Churn.csv')

# Visualizar los primeros registros
print(df.head())
print()
print(df.info())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

`El conjunto de datos proporcionado por Beta Bank contiene información de 10,000 clientes, con un total de 14 columnas que incluyen datos demográficos, financieros y de comportamiento. Entre las variables destacan el puntaje de crédito (CreditScore), país de residencia (Geography), género (Gender), edad (Age), años de permanencia (Tenure), saldo en cuenta (Balance), número de productos contratados (NumOfProducts), si posee tarjeta de crédito (HasCrCard), si es un cliente activo (IsActiveMember) y el salario estimado (EstimatedSalary). La variable objetivo es Exited, que indica si el cliente abandonó el banco (1) o no (0).`

# Preprocesamiento de los datos

In [3]:
# Eliminar las variables que no aportan valor descriptivo 
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis= 1)

In [4]:
# Llenar valores asuentes de la columna Tenure con la mediana 
df['Tenure'].fillna(df['Tenure'].median(), inplace = True)

In [5]:
# OHE para las variables categóricas : gender y geography 
df = pd.get_dummies(df, drop_first=True)

In [6]:
# Separar la varible objetivo y las características 
features = df.drop('Exited', axis = 1)
target = df['Exited']

In [7]:
# Dividimos entrenamiento + validación y prueba
fts_train_val, fts_test, tar_train_val, tar_test = train_test_split(features, target, test_size= 0.2, random_state = 54321)

# Ahora dividimos entre entrenamiento y validación 
fts_train, fts_valid, tar_train, tar_valid = train_test_split(fts_train_val, tar_train_val, test_size = 0.25, random_state = 54321)

In [8]:
# Se escalan las características
scaler = StandardScaler()
fts_train_scaled = scaler.fit_transform(fts_train)
fts_valid_scaled = scaler.transform(fts_valid)
fts_test_scaled = scaler.transform(fts_test)

`En esta etapa se prepararon los datos para el entrenamiento de modelos. Primero se eliminaron columnas irrelevantes como RowNumber, CustomerId y Surname, ya que no aportan información predictiva. Se imputaron los valores faltantes en la columna Tenure usando la mediana. Posteriormente, se aplicó codificación one-hot a las variables categóricas (Geography y Gender), eliminando una categoría para evitar multicolinealidad. La variable objetivo (Exited) se separó de las características. Luego, el conjunto de datos fue dividido en tres partes: entrenamiento (60%), validación (20%) y prueba (20%). Finalmente, se aplicó escalado de características numéricas con StandardScaler, garantizando que todas las variables tuvieran la misma escala para mejorar el rendimiento de los algoritmos.`

# Modelo Base (sin correción del desequilibrio)

In [9]:
# Entrenar el modelo 
model_base = RandomForestClassifier(random_state= 54321)
model_base.fit(fts_train_scaled, tar_train)

# Validación 
base_predicted = model_base.predict(fts_valid_scaled)

# Evaluación 
f1_base = f1_score(tar_valid, base_predicted)
roc_base = roc_auc_score(tar_valid, model_base.predict_proba(fts_valid_scaled)[:, 1])

print('Modelo Base:')
print("F1 Score (validación):", round(f1_base, 3))
print("AUC-ROC:", round(roc_base, 3))
print("Matriz de confusión:\n", confusion_matrix(tar_valid, base_predicted))

Modelo Base:
F1 Score (validación): 0.581
AUC-ROC: 0.852
Matriz de confusión:
 [[1485   73]
 [ 231  211]]


`El modelo base, entrenado sin aplicar ninguna técnica para corregir el desbalance de clases, alcanzó un F1 Score de 0.581 en el conjunto de validación, ligeramente por debajo del umbral requerido de 0.59. Sin embargo, obtuvo una AUC-ROC de 0.852, lo cual indica que el modelo tiene una muy buena capacidad para distinguir entre clientes que abandonan y los que no. La matriz de confusión muestra que el modelo predijo correctamente a 1,485 clientes que se quedaron y a 211 que se fueron. Sin embargo, también cometió 231 falsos negativos, es decir, clientes que realmente abandonaron pero el modelo no los detectó. Esto sugiere que, aunque el modelo es bueno en términos generales (AUC-ROC), tiende a favorecer la clase mayoritaria (clientes que se quedan), lo cual es un comportamiento esperado cuando hay desequilibrio en los datos y no se corrige.`

# Comprobar desequilibrio de clases 

In [10]:
print("Distribución de clases en todo el dataset:")
print(target.value_counts(normalize=True))

Distribución de clases en todo el dataset:
0    0.7963
1    0.2037
Name: Exited, dtype: float64


`Al analizar la variable objetivo (Exited), se observa un desequilibrio significativo en las clases: aproximadamente el 79.6% de los clientes permanecen en el banco (clase 0), mientras que solo el 20.4% lo abandonan (clase 1). Esta distribución desbalanceada puede afectar negativamente el rendimiento del modelo, ya que los algoritmos de clasificación tienden a favorecer la clase mayoritaria, resultando en una baja sensibilidad para detectar a los clientes que realmente se van. Por esta razón, es necesario aplicar técnicas de balanceo de clases durante el entrenamiento para mejorar la capacidad del modelo de identificar correctamente a los clientes en riesgo de abandono.`

# Correción del desbalance -> Opción #1 

In [11]:
# Entrenar de nuevo el modelo pero con la correción de clases opción 1 
model_weighted_1 = RandomForestClassifier(random_state = 54321, class_weight='balanced')
model_weighted_1.fit(fts_train_scaled, tar_train)

# Validación 
w_predicted_1 = model_weighted_1.predict(fts_valid_scaled)

# Evaluación 
f1_w_1 = f1_score(tar_valid, w_predicted_1)
roc_w_1 = roc_auc_score(tar_valid, model_weighted_1.predict_proba(fts_valid_scaled)[:, 1])

print('Modelo Balanceado Opción 1:')
print("F1 Score (validación):", round(f1_w_1, 3))
print("AUC-ROC:", round(roc_w_1, 3))
print("Matriz de confusión:\n", confusion_matrix(tar_valid, w_predicted_1))

Modelo Balanceado Opción 1:
F1 Score (validación): 0.544
AUC-ROC: 0.848
Matriz de confusión:
 [[1500   58]
 [ 255  187]]


`Al aplicar la primera técnica de corrección del desbalance mediante la asignación automática de pesos (class_weight='balanced'), el modelo logró un F1 Score de 0.544, ligeramente inferior al modelo base (0.581), y una AUC-ROC de 0.848, también apenas menor. La matriz de confusión muestra que el modelo detectó correctamente a 187 clientes que abandonaron el banco, pero aumentó el número de falsos negativos a 255. Aunque esta estrategia buscaba mejorar la detección de la clase minoritaria, en este caso no logró superar el rendimiento del modelo base, indicando que ajustar los pesos no fue suficiente para mejorar la capacidad del modelo de identificar a los clientes que se van.`

# Correción del desbalance -> Opción #2

In [12]:
# Submuestreo
def downsample(features, target, frac_majority):
    # Alinear índices
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # Separar clases
    majority_class = features[target == 0]
    minority_class = features[target == 1]
    majority_target = target[target == 0]
    minority_target = target[target == 1]

    # Submuestreo de clase mayoritaria
    majority_down = majority_class.sample(frac=frac_majority, random_state=54321)
    majority_target_down = majority_target.loc[majority_down.index]

    # Combinar datos balanceados
    features_downsampled = pd.concat([majority_down, minority_class])
    target_downsampled = pd.concat([majority_target_down, minority_target])

    # Mezclar
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=54321
    )

    return features_downsampled, target_downsampled

# Aplicar función
features_downsampled, target_downsampled = downsample(
    pd.DataFrame(fts_train_scaled, columns=fts_train.columns),
    tar_train,
    frac_majority=0.25
)

# Entrenar modelo
model_weighted_2 = RandomForestClassifier(random_state=54321)
model_weighted_2.fit(features_downsampled, target_downsampled)

# Validación
w_predicted_2 = model_weighted_2.predict(fts_valid_scaled)

# Evaluación
f1_w_2 = f1_score(tar_valid, w_predicted_2)
roc_w_2 = roc_auc_score(tar_valid, model_weighted_2.predict_proba(fts_valid_scaled)[:, 1])

print('Modelo Balanceado Opción 2:')
print("F1 Score (validación):", round(f1_w_2, 3))
print("AUC-ROC:", round(roc_w_2, 3))
print("Matriz de confusión:\n", confusion_matrix(tar_valid, w_predicted_2))

Modelo Balanceado Opción 2:
F1 Score (validación): 0.6
AUC-ROC: 0.854
Matriz de confusión:
 [[1216  342]
 [ 106  336]]


`Tras aplicar submuestreo para equilibrar las clases en el conjunto de entrenamiento, el modelo alcanzó un F1 Score de 0.60, superando el umbral mínimo requerido de 0.59, lo cual indica un buen equilibrio entre precisión y recall para la clase minoritaria. Además, obtuvo una AUC-ROC de 0.854, reflejando una muy buena capacidad de discriminación entre clases. La matriz de confusión muestra una mejora significativa en la detección de clientes que abandonan el banco (336 verdaderos positivos), con una reducción en los falsos negativos a 106, en comparación con modelos anteriores. Estos resultados sugieren que el submuestreo fue efectivo para mejorar la sensibilidad del modelo sin sacrificar mucho rendimiento general.`

# Comparación final y selección del mejor modelo

In [13]:
print("\nResumen de modelos en validación:")
print(f"Modelo base         - F1: {f1_base:.3f}, AUC: {roc_base:.3f}")
print(f"Class_weight        - F1: {f1_w_1:.3f}, AUC: {roc_w_1:.3f}")
print(f"Submuestreo manual  - F1: {f1_w_2:.3f}, AUC: {roc_w_2:.3f}")


Resumen de modelos en validación:
Modelo base         - F1: 0.581, AUC: 0.852
Class_weight        - F1: 0.544, AUC: 0.848
Submuestreo manual  - F1: 0.600, AUC: 0.854


`Se evaluaron tres enfoques distintos para el entrenamiento del modelo. El modelo base, sin corrección de desequilibrio, obtuvo un F1 Score de 0.581 y un AUC-ROC de 0.852, mostrando buena capacidad para distinguir clases, pero con tendencia a favorecer la clase mayoritaria. Al aplicar class_weight='balanced', el rendimiento del modelo disminuyó, con un F1 Score de 0.544 y AUC-ROC de 0.848, lo que sugiere que este ajuste no fue suficiente para mejorar la detección de clientes que abandonan. En cambio, el enfoque de submuestreo logró los mejores resultados, con un F1 Score de 0.600 y un AUC-ROC de 0.854, superando el umbral requerido. Este método mejoró el equilibrio entre precisión y recall, convirtiéndose en la mejor opción para predecir la pérdida de clientes.`

# Prueba final con el mejor modelo

In [14]:
# Probar el mejor modelo con el dataset de prueba
test_preds = model_weighted_2.predict(fts_test_scaled)

# Evaluación final 
f1_final = f1_score(tar_test, test_preds)
roc_final = roc_auc_score(tar_test, model_weighted_2.predict_proba(fts_test_scaled)[:, 1])

print('Modelo Balanceado Opción 2 con DataSet de Prueba')
print("F1 Score (validación):", round(f1_final, 3))
print("AUC-ROC:", round(roc_final, 3))
print("Matriz de confusión:\n", confusion_matrix(tar_test, test_preds))

Modelo Balanceado Opción 2 con DataSet de Prueba
F1 Score (validación): 0.605
AUC-ROC: 0.868
Matriz de confusión:
 [[1306  304]
 [  89  301]]


`El modelo seleccionado —entrenado con submuestreo— fue evaluado en el conjunto de prueba, alcanzando un F1 Score de 0.605, lo que confirma la consistencia y robustez del modelo, superando nuevamente el umbral mínimo requerido. Además, logró un AUC-ROC de 0.868, lo cual indica una muy buena capacidad de discriminación entre clientes que abandonan y los que permanecen. La matriz de confusión muestra que el modelo identificó correctamente a 301 clientes que abandonaron el banco y redujo los falsos negativos a solo 89, lo que es crucial para tomar acciones preventivas. Estos resultados validan que el modelo generaliza bien y es confiable para su uso en un entorno real.`

# Conclusión 

` En este proyecto se desarrolló un modelo de machine learning para predecir la pérdida de clientes del Beta Bank, utilizando un conjunto de datos históricos con diversas características demográficas y financieras. Durante el análisis exploratorio, se identificó un desequilibrio significativo en las clases, por lo que se implementaron distintas estrategias para corregirlo: entrenamiento sin corrección (modelo base), ajuste de pesos (class_weight='balanced') y submuestreo manual de la clase mayoritaria. `

`Tras comparar los resultados en el conjunto de validación, se determinó que el enfoque de submuestreo fue el más efectivo, al lograr el mejor balance entre precisión y recall, con un F1 Score de 0.600 y un AUC-ROC de 0.854. Finalmente, al evaluarlo en el conjunto de prueba, el modelo alcanzó un F1 Score de 0.605 y un AUC-ROC de 0.868, lo que demuestra su capacidad para generalizar y detectar correctamente a los clientes en riesgo de abandono.`

`En conclusión, se logró desarrollar un modelo predictivo eficaz, que no solo supera el umbral mínimo requerido, sino que también ofrece una herramienta útil para que el banco tome decisiones informadas y proactivas para reducir la pérdida de clientes. `